# Sistemas en tiempo continuo: Métodos numéricos para EDOs

Los sistemas dinámicos en tiempo continuo, en la forma de ecuaciones diferenciales ordinarias (EDOs) son comunes, en particular, como modelos que surgen desde las ciencias (química, física, biología, etc.).

Trabajaremos con EDOs homogéneas de la forma

$$\dot{\mathbf{x}} = \mathbf{f}_\mu(\mathbf{x})$$

con condición inicial $\mathbf{x}_\mu(t_0) = \mathbf{x}_{\mu,0}$.

Más explícitamente, tenemos
$$\dot{\mathbf{x}}(t) = \mathbf{f}_\mu(\mathbf{x}(t)) \quad \forall t.$$


La solución con una condición inicial dada es una función $t \mapsto \mathbf{x_\mu}(t)$, con $\mathbf{x}_\mu(t) \in \mathbb{R}^n$. La solución al tiempo $t$ empezando desde la condición inicial $x_0$ se llama el **flujo**, $\phi_\mu(t, \mathbf{x}_0)$.

## Soluciones numéricas de EDOs

Recordemos que hay teoremas que demuestran la existencia y unicidad de soluciones a una EDO bajo ciertas condiciones, por ejemplo si la función $\mathbf{f}$ es $C^1$.

Pero, ¿cómo podemos realmente calcular una solución a una EDO nolineal? Sabemos que la gran mayoría de las EDOs no tienen solución analítica. Aunque la teoría de los sistemas dinámicos provee métodos para analizar cualitativamente EDOs, quisiéramos poder calcular trayectorias, bifurcaciones, etc.

Para ello, debemos, entonces, encontrar una forma de integrar las ecuaciones de movimiento de un sistema *de forma numérica*. Cualquier solución numérica dará una *aproximación* a la función $\phi(t, \mathbf{x}_0)$. 

Hay muchos algoritmos para encontrar soluciones numéricas aproximadas a una EDO. Julia cuenta con una biblioteca [DifferentialEquations.jl](http://docs.juliadiffeq.org/stable/), que contiene un número muy grande de métodos para ecuaciones diferenciales de diferentes tipos (incluyendo ecuaciones estocásticas, EDPs, etc.). Además, el paquete [DynamicalSystems.jl](https://juliadynamics.github.io/DynamicalSystems.jl/latest/) provee un interfaz aún más bonito para sistemas dinámicos.

Sin embargo, aquí introduciremos una técnica muy poderosa que permite integrar EDOs con una precisión arbitrariamente alta, el llamado **método de Taylor**, implementado en [TaylorIntegration.jl](https://github.com/PerezHz/TaylorIntegration.jl).

## El método de Euler

Empezaremos con el método más sencillo de todos -- el cual nunca se debería utilizar -- para acercarnos al tema: el **método de Euler**.

Consideremos una EDO en una variable: $\dot{x} = f(x)$, con $x(0) = x_0$.

Una manera común de aproximar la solución continua $x(t)$ es **discretizar**la, es decir, buscar la solución a tiempos $t_n := n \, h$, con pasos de tiempo de tamaño pequeño $h$. Llamaremos $x_n$ la solución aproximada al tiempo $t_n$.

Cómo podemos tomar el primer paso, para aproximar $x(t=h)$ con $x_1$? Una idea es desarrollar en una serie de Taylor:

$$x(h) = x(0 + h) = x(0) + h \, \dot{x}(0) 
    + \frac{1}{2} h^2 \, \ddot{x}(0) + \cdots.$$
    
Dado que $\dot{x}(0) = f(x(0)) = f(x_0)$, obtenemos

$$x_1 = x_0 + h \, f(x_0) + \mathcal{O}(h^2).$$

Teniendo $x_1$, seguimos con el mismo método para obtener $x_2$, y así sucesivamente. En general, tenemos

$$x_{n+1} = x_n + h \, f(x_n),$$

lo cual es el método de Euler.


Para sistemas dinámicos de dimensión superior, esto se generaliza a

$$\mathbf{x}_{n+1} = \mathbf{x}_n + h \, \mathbf{f}(\mathbf{x}_n).$$


#### Ejercicio

[1] Implementa el método de Euler y aplícalo a la EDO $\dot{x} = -x$. Compara la solución gráficamente con la solución analítica para distintos valores de $h$.

[2] Utiliza el métodod de Euler para resolver el oscilador armónico, $\ddot{x} + x = 0$. Dibuja las soluciones. ¿Qué observas? ¿Por qué está equivocado?

Pista: Reescribe la EDO como dos ecuaciones de primer orden.

Por lo tanto, *nunca se debe utilizar el método de Euler en un cálculo real; sólo se muestra para fines pedagógicos*.

## Métodos de Runge-Kutta

Debemos mencionar a los métodos de Runge-Kutta, los cuales son, tal vez, los más utilizados. La idea es aproximar la solución con varios pasos de Euler para que la solución aproximada reproduzca la expansión de Taylor de la solución exacta hasta cierto orden, al evaluar la función $f$ en distintos lugares. 

Sin embargo, está muy difícil encontrar métodos de Runge-Kutta de orden alto.

## Método de Taylor

El método de Taylor extiende el método de Euler, al desarrollar en una serie de Taylor de orden superior. Pero nos topamos con un problema, e.g. a orden 2:

$$x(h) = x(0 + h) = x(0) + h \, \dot{x}(0) 
        + \frac{1}{2} h^2 \, \ddot{x}(0) + \mathcal{O}(h^3).$$
    
Para calcular $\ddot{x}(t)$, debemos derivar la EDO original, lo cual da

$$\ddot{x}(t) = \frac{d}{dt}[f(x(t))] = f'(x(t)) \cdot \dot{x}(t).$$

Por lo tanto, para poder utilizar este método, necesitamos poder calcular derivadas de la función $f$. Además, los términos de orden superior se ponen muy complicados. Por lo tanto, este método a menudo se descarta en los libros.

### Manipulando series de Taylor

Sin embargo, hay una solución bonita y eficaz, utilizando una extensión de la diferenciación automática a derivadas superiores.

La idea es desarrollar $x(t)$ en una serie de Taylor con coeficientes desconocidos, hasta cierto orden $n$:

$$x(t) = x_0 + x_1 \, t + x_2 \, t^2 + \cdots + x_n t^n.$$

[Estrictamente, esto es un **polinomio de Taylor**, no una serie de Taylor.]

Por lo tanto, el lado izquierdo de la EDO se vuelve

$$\dot{x}(t) = x_1 + 2 x_2 t + 3 x_3 t^2 + \cdots + n x_{n} t^{n-1}.$$


Del lado derecho de la EDO, tenemos $f(x(t))$. Lo que necesitamos es poder calcular la expansión de $f(x(t))$ como serie de Taylor. Para hacerlo, necesitamos reglas para cada función elemental $f$ que queramos utilizar, e.g. $x \mapsto x^2$, $x \mapsto \exp(x)$, etc.

Como ejemplo, consideremos la EDO

$$\dot{x} = x^2.$$

Para esto, necesitamos saber cómo multiplicar dos series de Taylor.

El producto de dos series de Taylor, $x(t)$ y $y(t) = \sum_{i=0} y_i t^i$. Si el resultado se llama $z = \sum_{n=0} z_n t^n$, podemos ver que el coeficiente $z_n$ está dada por

$$z_n = \sum_{i=0}^n x_i \, y_{n-i}.$$

#### Ejercicio

Escribe una función que calcule el coeficiente $z_n$, dado dos arreglos de coeficientes $x_i$ y $y_i$.

###  Resolviendo la EDO

Para la EDO $\dot{x} = f(x)$ con $f(x) = x^2$, ahora sabríamos cómo calcular los coeficientes del polinomio $f(x(t))$, si conociéramos los coeficientes de $x(t)$.

Pero, ¡justo son los coeficientes de $x(t)$ lo que queremos calcular! Parece que no hay solución a este problema... pero de hecho sí la hay, como sigue.

Desarrollemos $f(x(t)) = \sum_{i=0} f_i t^i$. Ya podemos calcular el término de orden $0$, es decir $f_0 = x_0^2$, ya que conocemos $x_0$.

Igualando coeficientes de $t^0$ por los dos lados de la EDO nos da $x_1 = f_0$, así que conocemos $x_1$.

Pero entonces ya podemos calcular $f_1 = 2 \, x_0 \, x_1$.


Vemos que, en general, $x_{n+1} = \frac{f_{n}}{n+1}$, y $f_n$ se puede calcular a partir de los coeficientes de $x(t)$ hasta orden $n$.

Por lo tanto, ¡podemos calcular todos los coeficientes de la solución $x(t)$ de forma recursiva! Esto es el "truco" que permite que el método de Taylor se pueda implementar en una computadora.

#### Ejercicio

Utiliza el método de Taylor para resolver $\dot{x} = x^2$. Compara con la solución analítica.

Nota que hemos hecho un cálculo "analítico" de forma numérica. Finalmente, los métodos analíticos a menudo se reducen a manipular polinomios, lo cual se puede implementar 

## TaylorIntegration.jl

El paquete TaylorIntegration.jl implementa este método. 

## Jet transport

A menudo queremos resolver la dinámica no solo de una condición inicial, sino también de condiciones iniciales cercanas.

Esto se puede hacer al sumarle a la condición inicial un polinomio en variables $\xi_1$, $\xi_2$, etc., e integrar también este polinomio. Esto se llama **jet transport**. (Ver notebook.)

## Integración rigurosa

Es posible combinar series de Taylor e intervalos para integrar EDOs de forma *rigurosa*. Ver e.g. Tucker, *Validated Numerics*.